In [ ]:
import numpy as np
from sklearn.linear_model import ElasticNet
from scipy.linalg import svd, eig

In [ ]:
def dmd(X1, X2, r=None):
    """
    X1, X2: 観測行列（X1: t=1~T-1, X2: t=2~T）
    r: 特異値分解のランク（省略可）
    """
    U, S, Vh = svd(X1, full_matrices=False)
    if r:
        U, S, Vh = U[:, :r], S[:r], Vh[:r, :]
    A_tilde = U.T @ X2 @ Vh.T @ np.linalg.inv(np.diag(S))
    eigvals, W = eig(A_tilde)
    Phi = X2 @ Vh.T @ np.linalg.inv(np.diag(S)) @ W
    return eigvals, Phi

In [ ]:
def adaptive_elastic_net(Phi, x0, lambd1=1.0, lambd2=1.0, weights=None):
    """
    Phi: モード行列 (r, N)
    x0: 初期値ベクトル (N,)
    lambd1, lambd2: L1, L2正則化パラメータ
    weights: 重み（adaptiveな場合は個別に設定）
    """
    enet = ElasticNet(alpha=lambd1+lambd2, l1_ratio=lambd1/(lambd1+lambd2), fit_intercept=False)
    if weights is not None:
        Phi = Phi / weights[:, None]
    enet.fit(Phi.T, x0)
    b = enet.coef_
    return b

In [ ]:
def aedmd_momentum_strategy(price_series, window=250, lambd1=1.0, lambd2=1.0):
    """
    price_series: (T, N) の株価系列（T: 時系列長、N: 資産数）
    window: サンプル数
    lambd1, lambd2: ElasticNetの正則化パラメータ
    """
    returns = np.diff(np.log(price_series), axis=0)
    signals = []
    for t in range(window, len(returns)):
        X1 = returns[t-window:t-1].T
        X2 = returns[t-window+1:t].T
        eigvals, Phi = dmd(X1, X2)
        x0 = returns[t-1]
        b = adaptive_elastic_net(Phi, x0, lambd1, lambd2)
        # 予測: ここではモードの寄与の大きいものをトレンド判定に使用
        pred = Phi @ b
        signals.append(np.sign(pred))
    return np.array(signals)

In [ ]:
def backtest(signals, returns):
    # signals, returns: shape=(期間, 資産数)
    port_ret = (signals * returns[-len(signals):]).mean(axis=1)
    mean = np.mean(port_ret)
    sd = np.std(port_ret)
    sr = mean / sd
    print(f'Mean: {mean:.4f}, SD: {sd:.4f}, SR: {sr:.4f}')
    return mean, sd, sr

In [ ]:
np.random.seed(0)
T, N = 1000, 5
price = np.cumprod(1 + 0.01 * np.random.randn(T, N), axis=0)
signals = aedmd_momentum_strategy(price, window=250, lambd1=0.1, lambd2=0.1)
rets = np.diff(np.log(price), axis=0)
mean, sd, sr = backtest(signals, rets)